# 🔋 배터리 데이터 전처리 및 분석 통합 노트북

## 📋 목차
1. 환경 설정 및 라이브러리 로드
2. 원시 데이터 전처리
3. 시계열 순서 정렬
4. 채널별 데이터 분리
5. 누적시간 기반 시각화
6. 사이클별 성능 분석
7. 통계 요약 및 리포트

## 📁 프로젝트 구조
```
DataPreprocess_250826/
├── src/                    # 소스코드
├── outputs/               # 출력 폴더
│   └── run_YYYYMMDD_HHMMSS/
│       ├── channels/      # 채널별 분리 데이터
│       ├── plots/         # 시각화 파일
│       ├── processed/     # 전처리된 원본 데이터
│       └── logs/          # 로그 파일
└── Rawdata/               # 원시 데이터
```

## 1. 환경 설정 및 라이브러리 로드

In [ ]:
# 필수 라이브러리 임포트
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime
import warnings
import logging

# 경고 메시지 무시
warnings.filterwarnings('ignore')

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# seaborn 스타일 설정
sns.set_style('whitegrid')
sns.set_palette('husl')

# 프로젝트 경로 설정
project_root = Path().absolute().parent  # src 폴더의 상위 폴더
sys.path.append(str(project_root / 'src'))

print("[환경설정] 완료")
print(f"[프로젝트 경로] {project_root}")
print(f"[Python 버전] {sys.version}")
print(f"[Pandas 버전] {pd.__version__}")
print(f"[Numpy 버전] {np.__version__}")

## 2. 원시 데이터 전처리

In [ ]:
# 배터리 프로세서 모듈 임포트
try:
    from improved_battery_processor import BatteryDataProcessor, Config
    print("[OK] 배터리 프로세서 모듈 로드 완료")
except ImportError as e:
    print(f"[오류] 모듈 로드 실패: {e}")
    print("src/ 폴더에 improved_battery_processor.py가 있는지 확인하세요.")

In [ ]:
# 원시 데이터 처리
raw_data_path = project_root / 'Rawdata' / '250207_250307_3_김동진_1689mAh_ATL Q7M Inner 2C 상온수명 1-100cyc'

if raw_data_path.exists():
    print(f"[데이터 발견] {raw_data_path.name}")
    print("[처리 시작] 배터리 데이터 전처리 중...")
    
    # 설정
    config = Config(
        MAX_FILES_PER_CHANNEL=500,
        PARALLEL_PROCESSING=True,
        MAX_WORKERS=2,
        LOG_LEVEL='INFO'
    )
    
    processor = BatteryDataProcessor(config)
    
    # 데이터 처리 실행
    try:
        results = processor.process_paths([raw_data_path])
        
        if results and results[0].channel_data:
            result = results[0]
            print("\n[처리 완료] 성공!")
            print(f"  - 데이터 포맷: {result.data_format}")
            print(f"  - 채널 수: {len(result.channel_data)}개")
            print(f"  - CSV 파일: {result.csv_file}")
            print(f"  - 플롯 파일: {result.plot_file}")
            
            # 전역 변수로 저장
            processed_result = result
            channel_data_dict = result.channel_data
        else:
            print("[오류] 데이터 처리 실패")
            
    except Exception as e:
        print(f"[오류] 처리 중 에러: {e}")
        import traceback
        print(traceback.format_exc())
else:
    print(f"[오류] 원시 데이터를 찾을 수 없습니다: {raw_data_path}")

## 3. 시계열 순서 정렬

In [ ]:
def sort_timeseries_data(df):
    """
    시계열 데이터를 Date + Time 기준으로 정렬
    """
    if 'Date' in df.columns and 'Time' in df.columns:
        print("[정렬] Date + Time 기준 정렬 중...")
        
        # DateTime 생성
        df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
        
        # 정렬 전 상태
        is_sorted_before = df['DateTime'].is_monotonic_increasing
        print(f"  정렬 전: {'이미 정렬됨' if is_sorted_before else '정렬 필요'}")
        
        # 정렬
        df_sorted = df.sort_values('DateTime').reset_index(drop=True)
        
        # 정렬 후 상태
        is_sorted_after = df_sorted['DateTime'].is_monotonic_increasing
        print(f"  정렬 후: {'정렬 완료' if is_sorted_after else '정렬 실패'}")
        
        # DateTime 컬럼 제거 (임시 컬럼)
        df_sorted = df_sorted.drop('DateTime', axis=1)
        
        return df_sorted
    else:
        print("[경고] Date 또는 Time 컬럼이 없습니다.")
        return df

# 채널별 데이터 정렬
if 'channel_data_dict' in locals():
    print("\n[시계열 정렬] 채널별 데이터 정렬 시작")
    
    sorted_channel_data = {}
    
    for channel, df in channel_data_dict.items():
        print(f"\n{channel} 정렬 중...")
        df_sorted = sort_timeseries_data(df)
        sorted_channel_data[channel] = df_sorted
        print(f"  데이터 크기: {len(df_sorted):,} 행")
        
        # 시간 범위 확인
        if 'Date' in df_sorted.columns:
            print(f"  기간: {df_sorted['Date'].min()} ~ {df_sorted['Date'].max()}")
    
    channel_data_dict = sorted_channel_data
    print("\n[정렬 완료] 모든 채널 시계열 정렬 완료")
else:
    print("[건너뛰기] 처리된 데이터가 없습니다.")

## 4. 채널별 데이터 분리 및 저장

In [ ]:
# 프로젝트 구조 관리
try:
    from setup_project_structure import ProjectStructureManager
    print("[OK] 프로젝트 구조 관리 모듈 로드")
except ImportError:
    print("[경고] setup_project_structure 모듈을 찾을 수 없습니다.")

# 출력 디렉토리 생성
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_dir = project_root / 'outputs' / f'run_{timestamp}'
channels_dir = output_dir / 'channels'
plots_dir = output_dir / 'plots'
processed_dir = output_dir / 'processed'

# 디렉토리 생성
for dir_path in [channels_dir, plots_dir, processed_dir]:
    dir_path.mkdir(parents=True, exist_ok=True)

print(f"[디렉토리 생성] {output_dir}")

# 채널별 데이터 저장
if 'channel_data_dict' in locals():
    print("\n[채널별 저장] 데이터 파일 생성 중...")
    
    saved_files = {}
    
    for channel, df in channel_data_dict.items():
        # 파일명 생성
        base_name = '250207_250307_3_김동진_1689mAh_ATL_Q7M_Inner_2C_상온수명'
        channel_filename = f"{base_name}_{channel}.csv"
        channel_path = channels_dir / channel_filename
        
        # CSV 저장
        df.to_csv(channel_path, index=False, encoding='utf-8-sig')
        saved_files[channel] = channel_path
        
        file_size = channel_path.stat().st_size / (1024*1024)  # MB
        print(f"  {channel}: {channel_filename} ({file_size:.1f}MB)")
    
    print(f"\n[저장 완료] {len(saved_files)}개 채널 파일 생성")
    
    # 전역 변수로 저장
    channel_files = saved_files
else:
    print("[건너뛰기] 저장할 채널 데이터가 없습니다.")

## 5. 누적시간 기반 시각화

In [ ]:
def add_cumulative_time(df):
    """
    누적시간 컬럼 추가
    """
    if 'Date' in df.columns and 'Time' in df.columns:
        df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
        start_time = df['DateTime'].min()
        df['Cumulative_Time_Hours'] = (df['DateTime'] - start_time).dt.total_seconds() / 3600.0
    elif 'Time_Sec' in df.columns:
        min_time = df['Time_Sec'].min()
        df['Cumulative_Time_Hours'] = (df['Time_Sec'] - min_time) / 3600.0
    else:
        df['Cumulative_Time_Hours'] = np.arange(len(df)) / 3600.0
    
    return df

# 누적시간 추가 및 시각화
if 'channel_data_dict' in locals():
    print("[누적시간] 컬럼 생성 중...")
    
    # 누적시간 추가
    for channel, df in channel_data_dict.items():
        df = add_cumulative_time(df)
        channel_data_dict[channel] = df
        max_hours = df['Cumulative_Time_Hours'].max()
        print(f"  {channel}: 누적시간 {max_hours:.1f} 시간 ({max_hours/24:.1f} 일)")
    
    # 시각화
    print("\n[시각화] 누적시간 기반 플롯 생성 중...")
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('배터리 성능 분석 - 누적시간 기준', fontsize=16, fontweight='bold')
    
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
    
    for i, (channel, df) in enumerate(channel_data_dict.items()):
        color = colors[i % len(colors)]
        
        # 데이터 샘플링 (성능을 위해)
        sample_rate = max(1, len(df) // 5000)
        df_sample = df.iloc[::sample_rate].copy()
        
        # 1. 전압
        axes[0, 0].plot(df_sample['Cumulative_Time_Hours'], df_sample['Voltage_V'],
                       color=color, alpha=0.8, linewidth=1.5, label=channel)
        
        # 2. 전류
        axes[0, 1].plot(df_sample['Cumulative_Time_Hours'], df_sample['Current_A'],
                       color=color, alpha=0.8, linewidth=1.5, label=channel)
        
        # 3. 용량
        if 'Capacity_Ah' in df_sample.columns:
            axes[1, 0].plot(df_sample['Cumulative_Time_Hours'], df_sample['Capacity_Ah'],
                           color=color, alpha=0.8, linewidth=1.5, label=channel)
        
        # 4. SOC
        if 'SOC_%' in df_sample.columns:
            axes[1, 1].plot(df_sample['Cumulative_Time_Hours'], df_sample['SOC_%'],
                           color=color, alpha=0.8, linewidth=1.5, label=channel)
    
    # 축 설정
    titles = ['전압 vs 누적시간', '전류 vs 누적시간', '용량 vs 누적시간', 'SOC vs 누적시간']
    ylabels = ['전압 (V)', '전류 (A)', '용량 (Ah)', 'SOC (%)']
    
    for ax, title, ylabel in zip(axes.flat, titles, ylabels):
        ax.set_title(title, fontsize=12, fontweight='bold')
        ax.set_xlabel('누적시간 (hours)', fontsize=10)
        ax.set_ylabel(ylabel, fontsize=10)
        ax.legend(loc='best')
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    # 저장
    plot_path = plots_dir / 'cumulative_time_analysis.png'
    plt.savefig(plot_path, dpi=300, bbox_inches='tight')
    print(f"[저장] {plot_path}")
    
    plt.show()
    
else:
    print("[건너뛰기] 시각화할 데이터가 없습니다.")

## 6. 사이클별 성능 분석

In [ ]:
# 사이클별 요약 통계 계산
def calculate_cycle_summary(df):
    """
    사이클별 요약 통계 계산
    """
    if 'TotalCycle' not in df.columns:
        return pd.DataFrame()
    
    summary_list = []
    
    for cycle in df['TotalCycle'].unique():
        cycle_data = df[df['TotalCycle'] == cycle]
        
        if len(cycle_data) < 10:
            continue
        
        summary = {
            'Cycle': int(cycle),
            'Data_Points': len(cycle_data),
            'Duration_Hours': cycle_data['Cumulative_Time_Hours'].max() - cycle_data['Cumulative_Time_Hours'].min() if 'Cumulative_Time_Hours' in cycle_data.columns else 0,
            'Voltage_Min': cycle_data['Voltage_V'].min(),
            'Voltage_Max': cycle_data['Voltage_V'].max(),
            'Voltage_Avg': cycle_data['Voltage_V'].mean(),
            'Current_Min': cycle_data['Current_A'].min() if 'Current_A' in cycle_data.columns else 0,
            'Current_Max': cycle_data['Current_A'].max() if 'Current_A' in cycle_data.columns else 0,
            'Capacity_Max': cycle_data['Capacity_Ah'].max() if 'Capacity_Ah' in cycle_data.columns else 0,
            'Temperature_Avg': cycle_data['Temperature_C'].mean() if 'Temperature_C' in cycle_data.columns else 0
        }
        
        summary_list.append(summary)
    
    return pd.DataFrame(summary_list)

# 채널별 사이클 요약
if 'channel_data_dict' in locals():
    print("[사이클 분석] 채널별 사이클 요약 계산 중...")
    
    cycle_summaries = {}
    
    for channel, df in channel_data_dict.items():
        summary_df = calculate_cycle_summary(df)
        cycle_summaries[channel] = summary_df
        print(f"\n{channel} 사이클 요약:")
        print(f"  총 사이클 수: {len(summary_df)}")
        
        if not summary_df.empty:
            print(f"  평균 용량: {summary_df['Capacity_Max'].mean():.3f} Ah")
            print(f"  평균 전압: {summary_df['Voltage_Avg'].mean():.3f} V")
            print(f"  평균 지속시간: {summary_df['Duration_Hours'].mean():.2f} 시간")
    
    # 사이클별 용량 변화 시각화
    if cycle_summaries:
        print("\n[시각화] 사이클별 용량 변화")
        
        fig, ax = plt.subplots(figsize=(12, 6))
        
        for channel, summary_df in cycle_summaries.items():
            if 'Capacity_Max' in summary_df.columns:
                ax.plot(summary_df['Cycle'], summary_df['Capacity_Max'],
                       'o-', label=channel, markersize=4, linewidth=1.5)
        
        ax.set_title('사이클별 최대 용량 변화', fontsize=14, fontweight='bold')
        ax.set_xlabel('사이클', fontsize=12)
        ax.set_ylabel('최대 용량 (Ah)', fontsize=12)
        ax.legend(loc='best')
        ax.grid(True, alpha=0.3)
        
        plt.tight_layout()
        
        # 저장
        plot_path = plots_dir / 'cycle_capacity_trend.png'
        plt.savefig(plot_path, dpi=300, bbox_inches='tight')
        print(f"[저장] {plot_path}")
        
        plt.show()
else:
    print("[건너뛰기] 분석할 데이터가 없습니다.")

## 7. 통계 요약 및 리포트

In [ ]:
# 전체 통계 요약
def generate_summary_report(channel_data_dict, cycle_summaries, output_dir):
    """
    통계 요약 리포트 생성
    """
    report_lines = []
    report_lines.append("=" * 80)
    report_lines.append("배터리 데이터 분석 리포트")
    report_lines.append(f"생성 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    report_lines.append("=" * 80)
    report_lines.append("")
    
    # 채널별 요약
    report_lines.append("1. 채널별 데이터 요약")
    report_lines.append("-" * 40)
    
    for channel, df in channel_data_dict.items():
        report_lines.append(f"\n{channel}:")
        report_lines.append(f"  - 데이터 포인트: {len(df):,}개")
        
        if 'Date' in df.columns:
            report_lines.append(f"  - 측정 기간: {df['Date'].min()} ~ {df['Date'].max()}")
        
        if 'Voltage_V' in df.columns:
            report_lines.append(f"  - 전압 범위: {df['Voltage_V'].min():.3f} ~ {df['Voltage_V'].max():.3f} V")
            report_lines.append(f"  - 평균 전압: {df['Voltage_V'].mean():.3f} ± {df['Voltage_V'].std():.3f} V")
        
        if 'Current_A' in df.columns:
            report_lines.append(f"  - 전류 범위: {df['Current_A'].min():.3f} ~ {df['Current_A'].max():.3f} A")
        
        if 'Capacity_Ah' in df.columns:
            report_lines.append(f"  - 최대 용량: {df['Capacity_Ah'].max():.3f} Ah")
        
        if 'Cumulative_Time_Hours' in df.columns:
            total_hours = df['Cumulative_Time_Hours'].max()
            report_lines.append(f"  - 총 측정 시간: {total_hours:.1f} 시간 ({total_hours/24:.1f} 일)")
    
    # 사이클 요약
    report_lines.append("")
    report_lines.append("2. 사이클 분석 요약")
    report_lines.append("-" * 40)
    
    for channel, summary_df in cycle_summaries.items():
        if not summary_df.empty:
            report_lines.append(f"\n{channel}:")
            report_lines.append(f"  - 총 사이클 수: {len(summary_df)}")
            report_lines.append(f"  - 평균 사이클 시간: {summary_df['Duration_Hours'].mean():.2f} 시간")
            report_lines.append(f"  - 초기 용량 (1-10 사이클): {summary_df.head(10)['Capacity_Max'].mean():.3f} Ah")
            report_lines.append(f"  - 최근 용량 (마지막 10 사이클): {summary_df.tail(10)['Capacity_Max'].mean():.3f} Ah")
            
            # 용량 감소율 계산
            if len(summary_df) > 10:
                initial_cap = summary_df.head(10)['Capacity_Max'].mean()
                final_cap = summary_df.tail(10)['Capacity_Max'].mean()
                retention = (final_cap / initial_cap * 100) if initial_cap > 0 else 0
                report_lines.append(f"  - 용량 유지율: {retention:.1f}%")
    
    # 파일 정보
    report_lines.append("")
    report_lines.append("3. 생성된 파일")
    report_lines.append("-" * 40)
    report_lines.append(f"  - 출력 디렉토리: {output_dir}")
    report_lines.append(f"  - 채널 데이터: {output_dir}/channels/")
    report_lines.append(f"  - 시각화 파일: {output_dir}/plots/")
    report_lines.append(f"  - 처리된 원본: {output_dir}/processed/")
    
    report_lines.append("")
    report_lines.append("=" * 80)
    report_lines.append("리포트 끝")
    
    return "\n".join(report_lines)

# 리포트 생성 및 저장
if 'channel_data_dict' in locals() and 'cycle_summaries' in locals():
    print("[리포트] 통계 요약 리포트 생성 중...")
    
    report_content = generate_summary_report(channel_data_dict, cycle_summaries, output_dir)
    
    # 리포트 출력
    print("\n" + report_content)
    
    # 파일로 저장
    report_path = output_dir / 'analysis_report.txt'
    with open(report_path, 'w', encoding='utf-8') as f:
        f.write(report_content)
    
    print(f"\n[저장] 리포트 파일: {report_path}")
else:
    print("[건너뛰기] 리포트 생성을 위한 데이터가 없습니다.")

## 8. 최종 결과 확인

In [ ]:
# 생성된 파일 목록 확인
print("[최종 확인] 생성된 파일 목록\n")

if output_dir.exists():
    # 채널 파일
    channel_files = list(channels_dir.glob('*.csv'))
    print(f"📁 채널 데이터 ({len(channel_files)}개):")
    for file in channel_files:
        size_mb = file.stat().st_size / (1024*1024)
        print(f"  - {file.name} ({size_mb:.1f}MB)")
    
    # 플롯 파일
    plot_files = list(plots_dir.glob('*.png'))
    print(f"\n📊 시각화 파일 ({len(plot_files)}개):")
    for file in plot_files:
        size_kb = file.stat().st_size / 1024
        print(f"  - {file.name} ({size_kb:.1f}KB)")
    
    # 리포트 파일
    report_files = list(output_dir.glob('*.txt'))
    print(f"\n📄 리포트 파일 ({len(report_files)}개):")
    for file in report_files:
        print(f"  - {file.name}")
    
    print(f"\n✅ 모든 파일이 다음 위치에 저장되었습니다:")
    print(f"   {output_dir}")
else:
    print("❌ 출력 디렉토리를 찾을 수 없습니다.")

## 🎯 완료!

### ✅ 수행된 작업
1. **원시 데이터 전처리**: TOYO 포맷 자동 감지 및 처리
2. **시계열 정렬**: Date + Time 기준 올바른 순서 정렬
3. **채널별 분리**: Ch30, Ch31 개별 CSV 파일 생성
4. **누적시간 분석**: 전체 테스트 기간 연속 시각화
5. **사이클 성능**: 사이클별 용량 변화 추적
6. **통계 리포트**: 종합 분석 결과 문서화

### 📁 출력 구조
```
outputs/run_YYYYMMDD_HHMMSS/
├── channels/              # 채널별 CSV 데이터
│   ├── *_Ch30.csv
│   └── *_Ch31.csv
├── plots/                 # 시각화 결과
│   ├── cumulative_time_analysis.png
│   └── cycle_capacity_trend.png
├── processed/             # 원본 처리 데이터
└── analysis_report.txt   # 분석 리포트
```

### 🚀 다음 단계
- 머신러닝 모델을 이용한 수명 예측
- 이상 탐지 알고리즘 적용
- 다중 테스트 비교 분석
- 자동화된 리포트 생성 시스템 구축